# The Perceptron

In [ ]:
import numpy as np

def sigmoid(x):
    '''
    ranges from  0 to 1
    '''
    return 1 / (1 + np.exp(-x))

def relu(x):
    '''
    ranges from  0 to 1
    '''
    return np.where(x > 0, x, 0)


class Perceptron:
    # instantiating the class, initializing the weights and bias 
    def __init__(self, num_inputs):
        self.w1 = np.random.random(num_inputs) * 0.01
        self.b1 = 1
    
    # feedforward pass
    def predict(self, X):
        # compute activation for input layer
        activation = np.dot(X, self.w1) + self.b1
        # non-linear transform
        fX = sigmoid(activation)
        # check threshold: for sigmoid and relu, use 0.5, for tanh, use 0
        y = np.where(fX >= 0.5, 1, -1)
        return y
    
    # training step
    def fit(self, train_data, train_labels, num_epochs=50):
        # iterate over epochs
        for epoch in range(1, num_epochs+1):
            print(epoch, end=' ')
            
            # predict with current parameters
            for (X, y) in zip(train_data, train_labels):
                pred_label = self.predict(X)

                # adjusting weights and bias
                if pred_label != y:
                    self.w1 = self.w1 + (X * y)
                    self.b1 = self.b1 + y
        
        print()
        return None

Let's get some simple text data to predict. We will us sparse vectors and limit ourselves to the top 1500 features.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2, SelectKBest
import pandas as pd

tfidf = TfidfVectorizer(ngram_range=(1,2), 
                        min_df=0.01, 
                        max_df=0.7, 
                        sublinear_tf=True, 
                        stop_words='english')

selector = SelectKBest(k=1500, score_func=chi2)

# read in files (already preprocessed)
train = pd.read_csv('../data/sa_train.csv')
dev = pd.read_csv('../data/sa_test.csv')

# transform and select training data
X = tfidf.fit_transform(train.clean_text)
y = train.output.map({'neg':-1, 'pos':1}).values
X = selector.fit_transform(X, y).todense().A

# transform and select dev data
Z = selector.transform(tfidf.transform(dev.clean_text)).todense().A
z = dev.output.map({'neg':-1, 'pos':1}).values

## Exercise 1

Create a Perceptron object and fit it on the data `X` and `y` data.

In [ ]:
# your code here
p = Perceptron(num_inputs=1500)
p.fit(X, y)

Check the performance on the dev set

In [ ]:
from sklearn.metrics import classification_report
predictions = p.predict(Z)
print(classification_report(z, predictions))

## Exercise 2

Add a learning rate to the Perceptron. Use a default of $0.25$. Check the effect on performance with various learning rates.

Hint: you will have to edit the `_init_` function and the update in the `fit` function.

In [ ]:
# your code here


## Variants

Implement two common variants of the perceptron that can improve performance. 

Either copy over your code from above and modify it to produce the variants, or implement a version that takes a parameter to decide what variant to use.

### Exercise 3: Averaged Perceptron

Often, the perceptron algorithm's final decision boundary on the training set will result in only mediocre performance on a test set. This is true both when the algorithm finds a line that perfectly separates the two classes (and thus obtains 100% accuracy on the training set) and when such a line cannot be found, perhaps because the classes are not linearly separable. 

When the classes are separable, the "perfect" line may pass too close to some instances, at the risk of misclassifying similar though not identical instances in the test set. In that case, we say the classifier is *overfit*, because it relies on quirks of the training set and cannot generalize to new instances. Ideally, the line should be some distance away from the instances, leaving a *margin*. 

When the classes are not separable, the perceptron algorithm still tries as hard as it can to find the line, even if it eventually must give up and return whatever the current line is. Thus the final decision boundary is not necessarily the best one seen during the training.  

Rather than taking the final weight vector to classify new instances, we can use information from the entire training process. One way is to classify each new instance with *all* weight vectors we produced during training and then see how often it ends up with one class or the other. This is called the *voted* perceptron, and it is slightly messy. A more practical approximation is the **averaged** perceptron, which you will implement, where we use the average overall different weight vectors we have seen in training.

Hint: Save *a copy* of the current weight vector, including bias term, each time the weight vector is updated. When the training process has been completed, produce a new weight vector as the average of all saved vectors.

In [ ]:
# your code here

### Exercise 4: Pocket Perceptron

Another variant is the **pocket** perceptron, which keeps the best weight vector it has seen during training in its "pocket" and returns that as the final weight vector.

Hint: "Best" here means the weight vector with the highest classification accuracy on the training set. You need to implement a variant of `fit` that takes a dev set, its gold labels, and a scoring fucntion. 

In [ ]:
from sklearn.metrics import f1_score

# your code here

### Optional: Exercise 4

So far, we have used a version of the **online** perceptron, which updates after each instance. This results in a lot of updates per epoch, and can be sensitive to the order of instances.
Rather than updateing after *every* instance, we can update after $k$ instances. This variant is called the **batch** perceptron. If $k=1$, we have online learning, with $k=|X|$, we updates once after we have seen all instances. 

Change the `fit()` function to only update every $k$ instances. Instead of feature vectors, you can now operate on a matrix.

In [ ]:
# your code here